<a href="https://colab.research.google.com/github/jkf87/Midm-2.0/blob/main/Midm_2_0_Mini_Instruct_gradio_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch transformers accelerate

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

model_name = "K-intelligence/Midm-2.0-Mini-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
generation_config = GenerationConfig.from_pretrained(model_name)

prompt = "KT에 대해 소개해줘"

# message for inference
messages = [
    {"role": "system",
     "content": "Mi:dm(믿:음)은 KT에서 개발한 AI 기반 어시스턴트이다."},
    {"role": "user", "content": prompt}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

output = model.generate(
    input_ids.to("cuda"),
    generation_config=generation_config,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=128,
    do_sample=False,
)
print(tokenizer.decode(output[0]))


In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import torch
import re

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

def clean_response(response):
    """응답에서 특별한 토큰들을 제거하고 정리"""
    # 특별한 토큰들 제거
    tokens_to_remove = [
        '<|begin_of_text|>', '<|start_header_id|>', '<|end_header_id|>',
        '<|eot_id|>', 'system', 'user', 'assistant'
    ]

    for token in tokens_to_remove:
        response = response.replace(token, '')

    # 연속된 공백 제거
    response = re.sub(r'\s+', ' ', response)

    # 시스템 메시지 부분 제거 (Cutting Knowledge Date부터 시작하는 부분)
    if 'Cutting Knowledge Date:' in response:
        parts = response.split('KT에 대해 소개해줘')
        if len(parts) > 1:
            response = parts[-1].strip()

    return response.strip()

def chat_with_model(message, history):
    # 시스템 메시지 (간단하게 설정)
    messages = [
        {"role": "system",
         "content": "Mi:dm(믿:음)은 KT에서 개발한 AI 기반 어시스턴트이다."}
    ]

    # 히스토리 추가
    for user_msg, bot_msg in history:
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": bot_msg})

    # 현재 메시지 추가
    messages.append({"role": "user", "content": message})

    # 토크나이징 및 생성
    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    # attention mask 설정
    attention_mask = torch.ones_like(input_ids)

    # pad token 설정
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    with torch.no_grad():
        output = model.generate(
            input_ids.to(device),
            attention_mask=attention_mask.to(device),
            generation_config=generation_config,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
            max_new_tokens=512,  # 더 길게 설정
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
        )

    # 응답 추출 및 정리
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # 입력 부분 제거
    input_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    if input_text in response:
        response = response.replace(input_text, "").strip()

    # 응답 정리
    response = clean_response(response)

    # 빈 응답 처리
    if not response:
        response = "죄송합니다. 적절한 응답을 생성하지 못했습니다. 다시 시도해주세요."

    return response

# Gradio 인터페이스 생성
with gr.Blocks(theme=gr.themes.Soft(), title="Mi:dm AI 어시스턴트") as demo:
    gr.HTML("""
    <div style="text-align: center; margin-bottom: 20px;">
        <h1>🤖 Mi:dm (믿:음)</h1>
        <h3>KT AI 어시스턴트</h3>
        <p>KT에서 개발한 AI 기반 어시스턴트와 대화해보세요!</p>
    </div>
    """)

    chatbot = gr.Chatbot(
        height=500,
        show_label=False,
        container=True,
        show_copy_button=True
    )

    with gr.Row():
        msg = gr.Textbox(
            label="메시지",
            placeholder="안녕하세요! 무엇을 도와드릴까요?",
            lines=2,
            max_lines=10,
            scale=4
        )
        submit_btn = gr.Button("전송", variant="primary", scale=1)

    with gr.Row():
        clear_btn = gr.Button("대화 초기화", variant="secondary")

    # 예시 질문들
    with gr.Row():
        examples = [
            "안녕하세요! 자기소개 부탁드려요.",
            "KT에 대해 자세히 소개해주세요.",
            "KT의 주요 사업 영역은 무엇인가요?",
            "5G 기술에 대해 설명해주세요."
        ]

        for example in examples:
            gr.Button(example, size="sm").click(
                lambda x=example: (x, []),
                outputs=[msg, chatbot]
            )

    def respond(message, chat_history):
        if not message.strip():
            return chat_history, ""

        try:
            bot_response = chat_with_model(message, chat_history)
            chat_history.append([message, bot_response])
        except Exception as e:
            error_msg = f"오류가 발생했습니다: {str(e)}"
            chat_history.append([message, error_msg])

        return chat_history, ""

    def clear_chat():
        return []

    # 이벤트 연결
    submit_btn.click(
        respond,
        inputs=[msg, chatbot],
        outputs=[chatbot, msg]
    )

    msg.submit(
        respond,
        inputs=[msg, chatbot],
        outputs=[chatbot, msg]
    )

    clear_btn.click(
        clear_chat,
        outputs=[chatbot]
    )

# 실행
demo.launch(share=True, debug=True)